In [1]:
import pandas as pd
import ast

c:\Users\n9810200\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\n9810200\AppData\Local\Temp/ipykernel_405976/4135976894.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
denominator = 500 #same as the number of unique o2o relationships

##### Dirigo

In [22]:
dirigo_o2o = pd.read_csv('../Dirigo/OCEL_O2O.csv')
dirigo_o2o

,Source_object_id,Target_object_id,O2O_qualifier,Timestamp
0,Pcp44,Cr2,Lodged pickup plan for cargo,2024-04-29 08:04:44
1,Pcp87,Cr9,Lodged pickup plan for cargo,2024-04-29 08:12:32
2,Pcp48,Cr20,Lodged pickup plan for cargo,2024-04-29 08:31:44
3,Pcp55,Cr13,Lodged pickup plan for cargo,2024-04-29 08:32:30
4,Pcp69,Cr7,Lodged pickup plan for cargo,2024-04-29 08:39:46
...,...,...,...,...
495,tr29,Pcp51,Dropped truck from pickup plan,2024-05-02 15:18:40
496,tr29,Pcp70,Assigned truck for pickup plan,2024-05-03 15:08:59
497,tr49,Pcp70,Assigned truck for pickup plan,2024-05-03 15:09:23
498,tr49,Pcp70,Dropped truck from pickup plan,2024-05-03 15:38:46


In [27]:
qualified_o2o_dirigo = dirigo_o2o['O2O_qualifier'].notna() & (dirigo_o2o['O2O_qualifier'].str.strip() != '')
num_qualified_dirigo = qualified_o2o_dirigo.sum()
num_qualified_dirigo

500

In [31]:
o2oscr_dirigo = num_qualified_dirigo / denominator
print(f"O2O Qualifier Completeness Ratio (Dirigo): {o2oscr_dirigo:.2%}")


O2O Qualifier Completeness Ratio (Dirigo): 100.00%


##### ACEL

In [9]:
acel_events = pd.read_csv('../ACEL/ACEL_Events.csv')
acel_relations = pd.read_csv('../ACEL/ACEL_Relations.csv')

In [11]:
relation_type_map = acel_relations.set_index('RelationId')['Type'].to_dict()
relation_type_map

{'Cr2_to_Pcp44': 'is lodged to pick up',
 'Cr9_to_Pcp87': 'is lodged to pick up',
 'Cr20_to_Pcp48': 'is lodged to pick up',
 'Cr13_to_Pcp55': 'is lodged to pick up',
 'Cr7_to_Pcp69': 'is lodged to pick up',
 'Cr1_to_Pcp18': 'is lodged to pick up',
 'Cr8_to_Pcp21': 'is lodged to pick up',
 'Cr19_to_Pcp74': 'is lodged to pick up',
 'Cr17_to_Pcp23': 'is lodged to pick up',
 'Cr12_to_Pcp39': 'is lodged to pick up',
 'Cr15_to_Pcp41': 'is lodged to pick up',
 'Cr10_to_Pcp29': 'is lodged to pick up',
 'Cr11_to_Pcp36': 'is lodged to pick up',
 'Cr14_to_Pcp81': 'is lodged to pick up',
 'Cr5_to_Pcp22': 'is lodged to pick up',
 'Cr20_to_Pcp28': 'is lodged to pick up',
 'Cr15_to_Pcp59': 'is lodged to pick up',
 'Cr16_to_Pcp7': 'is lodged to pick up',
 'Cr10_to_Pcp64': 'is lodged to pick up',
 'Cr11_to_Pcp4': 'is lodged to pick up',
 'Cr2_to_Pcp5': 'is lodged to pick up',
 'Cr18_to_Pcp82': 'is lodged to pick up',
 'Cr9_to_Pcp98': 'is lodged to pick up',
 'Cr17_to_Pcp86': 'is lodged to pick up',
 'C

In [13]:
semantic_tuples = []

for _, row in acel_events.iterrows():
    timestamp = row['Timestamp']  
    relation_changes_raw = row.get('RelationChanges') 

    if pd.notna(relation_changes_raw):  
        try:
            relation_changes = ast.literal_eval(relation_changes_raw)

            
            for change in relation_changes:
                relation_id = change.get('RelationId')
                target = change.get('Target')
                qualifier = relation_type_map.get(relation_id)
                source = relation_id.split('_to_')[0] if '_to_' in relation_id else None

        
                if source and target and qualifier and timestamp:
                    semantic_tuples.append((source, target, qualifier, timestamp))

        except Exception as e:
            continue 


In [16]:
df_o2o_semantics = pd.DataFrame(
    semantic_tuples,
    columns=['Source_Object', 'Target_Object', 'Qualifier', 'Timestamp']
).drop_duplicates()
df_o2o_semantics 


,Source_Object,Target_Object,Qualifier,Timestamp
0,Cr2,Pcp44,is lodged to pick up,2024-04-29 08:04:44
1,Cr9,Pcp87,is lodged to pick up,2024-04-29 08:12:32
2,Cr20,Pcp48,is lodged to pick up,2024-04-29 08:31:44
3,Cr13,Pcp55,is lodged to pick up,2024-04-29 08:32:30
4,Cr7,Pcp69,is lodged to pick up,2024-04-29 08:39:46
...,...,...,...,...
495,Pcp51,tr29,is dropped from,2024-05-02 15:18:40
496,Pcp70,tr29,is dropped from,2024-05-03 15:08:59
497,Pcp70,tr49,is dropped from,2024-05-03 15:09:23
498,Pcp70,tr49,is dropped from,2024-05-03 15:38:46


In [29]:
qualified_o2o_acel = df_o2o_semantics ['Qualifier'].notna() & (df_o2o_semantics ['Qualifier'].str.strip() != '')
num_qualified_acel = qualified_o2o_acel.sum()
num_qualified_acel

500

In [30]:
o2oscr_acel = num_qualified_acel / denominator
print(f"O2O Qualifier Completeness Ratio (ACEL): {o2oscr_acel:.2%}")


O2O Qualifier Completeness Ratio (ACEL): 100.00%
